<a href="https://colab.research.google.com/github/Alexandre77777/data_mining/blob/main/8.%20%D0%90%D0%BD%D1%81%D0%B0%D0%BC%D0%B1%D0%BB%D0%B5%D0%B2%D1%8B%D0%B5%20%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B%20%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE%20%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8F/%D0%A2%D0%B5%D0%BE%D1%80%D0%B8%D1%8F%20%E2%84%9611.%20%D0%90%D0%BD%D1%81%D0%B0%D0%BC%D0%B1%D0%BB%D0%B5%D0%B2%D1%8B%D0%B5_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B_%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CatBoost vs Light GBM vs XGBoost vs AdaBoost

Cравнение четырех мощных алгоритмов градиентного бустинга: CatBoost, LightGBM, XGBoost и AdaBoost.

Эти алгоритмы завоевали огромную популярность благодаря своей эффективности в широком спектре областей - от задач классификации и регрессии до обнаружения аномалий и рекомендательных систем. На основе систематической оценки их достоинств и недостатков мы рассмотрим ключевые факторы, влияющие на их эффективность, такие как скорость обучения, точность предсказания и устойчивость к переобучению.

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
from time import time
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
!git clone https://github.com/tester170/Other.git

fatal: destination path 'Other' already exists and is not an empty directory.


In [ ]:
data = pd.read_csv('/content/Other/сollege_data.csv') # Описание: https://www.kaggle.com/datasets/yashgpt/us-college-data

data.loc[data.grad_rate > 100, 'grad_rate'] = 100

accuracy = {}
speed = {}

X = data.drop('private', axis=1)
y = data.private.map({"Yes": 1, "No": 0})

In [ ]:
data.private.value_counts()

Yes    565
No     212
Name: private, dtype: int64

# 1. `GradientBoostingClassifier` из Scikit-Learn

Документация: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

Классификатор Gradient Boosting Classifier - это метод машинного обучения, используемый для решения задач классификации. Это метод ансамблевого обучения, который объединяет прогнозы нескольких слабых моделей для получения сильного общего прогноза. Слабые модели, как правило, представляют собой деревья решений, и их прогнозы объединяются с помощью средневзвешенного значения, при этом больший вес имеют те деревья, которые дают лучшие результаты на обучающих данных. Веса обновляются на каждой итерации процесса усиления, в результате чего внимание уделяется образцам, которые были неправильно классифицированы на предыдущей итерации. Окончательный прогноз делается путем взвешенного большинства голосов всех деревьев в ансамбле.

Дополнительно: https://habr.com/ru/companies/rosbank/articles/747816/

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

model = GradientBoostingClassifier()

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1)

speed['GradientBoosting'] = np.round(time() - start, 3)
accuracy['GradientBoosting'] = (np.mean(score) * 100).round(3)

print(f"Mean F1 score: {accuracy['GradientBoosting']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['GradientBoosting']}s")

Mean F1 score: 95.871
STD: 0.014
Run Time: 19.365s


Этот код использует модель градиентного бустинга (GradientBoostingClassifier) из библиотеки scikit-learn для классификации данных. Он также использует кросс-валидацию и повторяющуюся стратифицированную K-кратную перекрестную проверку (RepeatedStratifiedKFold) для оценки производительности модели на наборе данных.

1. Импортируются необходимые библиотеки:
   - from sklearn.ensemble import GradientBoostingClassifier: импортируется классификатор градиентного бустинга из scikit-learn.
   - from sklearn.model_selection import cross_val_score: импортируется функция для вычисления оценки производительности с использованием кросс-валидации.
   - from sklearn.model_selection import RepeatedStratifiedKFold: импортируется функция для выполнения повторяющейся стратифицированной K-кратной перекрестной проверки.

2. Создается экземпляр модели GradientBoostingClassifier и сохраняется в переменной model.

3. Запоминается текущее время с помощью функции time() из модуля time.

4. Создается объект RepeatedStratifiedKFold с параметрами: n_splits=5 (количество разбиений), n_repeats=5 (количество повторений), random_state=42 (значение для генерации случайных чисел).

5. Выполняется кросс-валидация с использованием функции cross_val_score. Она принимает следующие аргументы:
   - model: экземпляр модели GradientBoostingClassifier.
   - X: матрица признаков (независимые переменные).
   - y: вектор целевых переменных.
   - scoring='f1': метрика для оценки производительности модели (в данном случае F1-мера).
   - cv: объект RepeatedStratifiedKFold для кросс-валидации.
   - n_jobs=-1: количество параллельных задач для выполнения (в данном случае используется все доступные ядра процессора).

6. Рассчитывается время выполнения кросс-валидации путем вычитания текущего времени из времени, сохраненного ранее. Затем результат округляется до трех знаков после запятой и сохраняется в словаре speed с ключом 'GradientBoosting'.

7. Рассчитывается средняя F1-мера, умноженная на 100 и округленная до трех знаков после запятой, с использованием np.mean(score) * 100. Результат сохраняется в словаре accuracy с ключом 'GradientBoosting'.

8. Выводится средняя F1-мера, используя значение из словаря accuracy с ключом 'GradientBoosting'.

9. Выводится стандартное отклонение оценок производительности модели, используя np.std(score) для вычисления стандартного отклонения.

10. Выводится время выполнения кросс-валидации, используя значение из словаря speed с ключом 'GradientBoosting'.

**Данный подход будет спользоваться на протяжении всего эксперимента**

# 2. XGBoost

XGBoost (eXtreme Gradient Boosting) - это оптимизированная и масштабируемая реализация градиентного бустинга для моделей на основе деревьев. Она отличается высокой эффективностью и производительностью и широко используется для решения крупномасштабных задач машинного обучения, таких как классификация и регрессия.

XGBoost отличают возможности параллельной обработки, способность работать с отсутствующими данными и расширенные возможности настройки модели. Кроме того, он предоставляет гибкий и выразительный синтаксис для определения моделей, что делает его доступным как для начинающих, так и для опытных пользователей.

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier()

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1)

speed['XGBoost'] = np.round(time() - start, 3)
accuracy['XGBoost'] = (np.mean(score) * 100).round(3)

print(f"Mean F1 score: {accuracy['XGBoost']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['XGBoost']}s")

Mean F1 score: 95.668
STD: 0.014
Run Time: 5.878s


# 3. LightGBM

Документация: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

LightGBM - это фреймворк градиентного бустинга, использующий древовидные алгоритмы обучения. Он разработан для эффективного и масштабируемого решения крупномасштабных задач машинного обучения, таких как классификация и регрессия. Особенно хорошо он подходит для работы с большими наборами данных благодаря высокой скорости обучения, низкому потреблению памяти и возможности работы с отсутствующими значениями.

Для работы с большими наборами данных в LightGBM используется новая градиентная техника односторонней выборки, которая позволяет достичь высокой скорости обучения при сохранении точности. Кроме того, в LightGBM реализованы расширенные функциональные возможности для настройки модели, такие как поддержка параллельных вычислений и вычислений на базе GPU, а также различные функции цели для задач бинарной и мультиклассовой классификации.

Дополнительно: https://habr.com/ru/companies/skillfactory/articles/530594/

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier()

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1)

speed['LGBM'] = np.round(time() - start, 3)
accuracy['LGBM'] = (np.mean(score) * 100).round(3)

print(f"Mean F1 score: {accuracy['LGBM']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['LGBM']}s")

[LightGBM] [Info] Number of positive: 452, number of negative: 169
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000292 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2323
[LightGBM] [Info] Number of data points in the train set: 621, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.727858 -> initscore=0.983783
[LightGBM] [Info] Start training from score 0.983783
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

# 4. CatBoost

Документация: https://yandex.ru/dev/catboost/

CatBoost - это библиотека градиентного бустинга, хорошо подходящая для работы с категориальными данными. Это библиотека с открытым исходным кодом, разработанная компанией Yandex и рассчитанная на быстрое масштабирование при работе с большими массивами данных.

Одной из главных особенностей CatBoost является возможность работы с категориальными признаками без необходимости однократного кодирования. Это достигается за счет использования упорядоченного кодирования категориальных признаков и учета информации о порядке в процессе обучения. Такой подход позволяет получить более точные модели и уменьшить перегрузку по сравнению с традиционными методами однократного кодирования.

CatBoost также включает расширенные возможности для подстройки модели, такие как поддержка параллельных вычислений, различные функции потерь и эффективная реализация алгоритма градиентного бустинга. Кроме того, CatBoost предоставляет простой и интуитивно понятный API для определения и обучения моделей, что делает его доступным как для начинающих, так и для опытных пользователей.

Дополнительно:

1. https://habr.com/ru/companies/otus/articles/527554/

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(silent=True)

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1)

speed['CatBoost'] = np.round(time() - start, 3)
accuracy['CatBoost'] = (np.mean(score) * 100).round(3)


print(f"Mean F1 score: {accuracy['CatBoost']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['CatBoost']}s")

Mean F1 score: 96.181
STD: 0.010
Run Time: 131.583s


# 5. AdaBoost

Документация: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

AdaBoost (Adaptive Boosting) - это метод машинного обучения, используемый для решения задач классификации и регрессии. Это метод ансамблевого обучения, который объединяет прогнозы нескольких слабых моделей для получения сильного общего прогноза. Слабые модели обычно представляют собой деревья решений с малой глубиной, а прогнозы объединяются с помощью средневзвешенного значения, при этом больший вес имеют деревья, дающие лучший результат на обучающих данных.

AdaBoost работает путем итеративного обучения слабых моделей на обучающих данных и корректировки весов на каждой итерации, чтобы сосредоточиться на образцах, которые были неправильно классифицированы на предыдущей итерации. Окончательный прогноз делается путем взвешенного большинства голосов всех деревьев в ансамбле.

Дополнительно: https://neerc.ifmo.ru/wiki/index.php?title=%D0%91%D1%83%D1%81%D1%82%D0%B8%D0%BD%D0%B3,_AdaBoost&mobileaction=toggle_view_desktop

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier()

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1)

speed['AdaBoost'] = np.round(time() - start, 3)
accuracy['AdaBoost'] = (np.mean(score) * 100).round(3)

print(f"Mean F1 score: {accuracy['AdaBoost']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['AdaBoost']}s")

Mean F1 score: 95.515
STD: 0.010
Run Time: 4.298s


# 6. Scikit-Learn vs XGBoost vs LightGBM vs CatBoost

In [ ]:
for algo, result in accuracy.items():
    print(f"{algo:{20}}: Score: {result}, Speed: {speed[algo]}")

GradientBoosting    : Score: 95.871, Speed: 19.365
XGBoost             : Score: 95.668, Speed: 5.878
LGBM                : Score: 95.916, Speed: 4.139
CatBoost            : Score: 96.181, Speed: 131.583
AdaBoost            : Score: 95.515, Speed: 4.298


In [ ]:
accuracy_df = pd.DataFrame(list(accuracy.items()), columns=['Algorithm', 'Accuracy'])

speed_df = pd.DataFrame(list(speed.items()), columns=['Algorithm', 'Time'])

In [ ]:
fig = px.bar(accuracy_df, y='Accuracy', x='Algorithm', text_auto='.5s',
            title="Accuracy")
fig.show()

In [ ]:
fig = px.bar(speed_df, y='Time', x='Algorithm', text_auto='.5s',
            title="Time")
fig.show()

# 7. Выводы

Результаты работы алгоритмов градиентного бустинга - GradientBoosting, XGBoost, LightGBM, CatBoost и AdaBoost - позволяют сделать ряд интересных выводов. Прежде всего, очевидно, что все эти алгоритмы работают достаточно хорошо, их точность варьируется от 95,515% до 96,181%. Это говорит о том, что градиентный бустинг в целом является эффективным методом.

По точности прогнозирования лидирует CatBoost, получивший впечатляющую оценку точности 96,181%. За ним следуют LightGBM с 95,916% и XGBoost с 95,856%. GradientBoosting и AdaBoost, несмотря на высокие результаты, демонстрируют несколько более низкие показатели точности - 95,856% и 95,515% соответственно.

Однако важно учитывать и скорость обучения. Здесь CatBoost значительно отстает от других алгоритмов, время обучения составляет 91,02 единицы. Самым быстрым оказался LightGBM, время обучения которого составило всего 2,686 единиц, за ним следуют XGBoost - 4,22 единицы и AdaBoost - 1,492 единицы. GradientBoosting находится в середине по скорости обучения, занимая 5,007 единиц.

Таким образом, хотя CatBoost и обеспечивает наивысшую точность прогнозирования, его использование связано со значительным увеличением времени обучения. LightGBM и XGBoost обеспечивают хороший баланс между точностью и скоростью, что делает их привлекательными для многих практических приложений. AdaBoost, несмотря на то, что является самым быстрым, несколько отстает по точности. При выборе оптимального алгоритма следует учитывать конкретные требования задачи: CatBoost может быть предпочтительнее, когда точность имеет первостепенное значение, а время не является ограничением, в то время как LightGBM или XGBoost могут быть более подходящими для приложений реального времени или крупных приложений, где скорость обучения является критичной.